В Базе Данных есть таблица pulsar_stars, в которой содержатся сведения о звездах, полученные в ходе исследовании вселенной (High Time Resolution Universe Survey) с целью определения одного из типа нейтронных звезд — пульсаров. Поле  TARGET таблицы  pulsar_stats является откликом, все остальные поля — предикторы.

Вам необходимо получить выборку из таблицы pulsar_stats с помощью запросов на основании следующих критериев:

- Все строки таблицы, где TARGET = 0 и MIP от 96.921875 до 97.4140625 включительно
- Все строки таблицы, где TARGET = 1 и MIP от 41.8828125 до 47.4140625 включительно




Число строк в полученной выборке

SELECT COUNT(*) FROM pulsar_stars WHERE ((TARGET=0 AND MIP between 96.921875 AND 97.4140625) OR (TARGET=1 AND MIP between 41.8828125 AND 47.4140625))

Строк: 202

In [18]:
#импортируем данные сюда
import numpy as np 
import pandas as pd 

data = pd.read_csv("data_stars.csv", delimiter=',')
data

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC,TARGET
0,97.234375,46.536357,0.558616,0.384429,2.010033,13.634376,10.022346,132.125623,0
1,42.578125,46.638337,2.841558,7.832819,12.971572,38.498849,3.799728,14.990834,1
2,97.171875,45.656172,0.721308,0.482644,1.557692,12.590775,11.434676,169.923103,0
3,42.023438,33.027088,3.668886,19.648005,87.904682,86.479951,0.259049,-1.504341,1
4,46.171875,36.158313,3.294613,13.590332,14.005017,43.687473,3.424031,11.564085,1
...,...,...,...,...,...,...,...,...,...
197,47.031250,36.980979,3.744839,16.509533,71.880435,73.559647,0.973483,-0.141037,1
198,97.148438,37.160145,0.457632,1.767783,2.464883,17.646414,9.645039,104.454324,0
199,97.101562,46.205864,0.486238,0.375747,0.418060,8.618880,25.031765,682.519179,0
200,44.796875,44.724043,3.508989,12.290881,57.566890,66.830025,1.231090,0.754103,1


Выборочное среднее для столбца MIP

In [3]:
data['MIP'].mean()

70.79517326732673

Линейная нормировка всех значений предикторов полученной выборки

In [52]:
from sklearn import preprocessing
data_stars = data.drop('TARGET', axis=1)
data_stars['MIP'] = preprocessing.minmax_scale(data_stars['MIP'])
data_stars['STDIP'] = preprocessing.minmax_scale(data_stars['STDIP'])
data_stars['EKIP'] = preprocessing.minmax_scale(data_stars['EKIP'])
data_stars['SIP'] = preprocessing.minmax_scale(data_stars['SIP'])
data_stars['MC'] = preprocessing.minmax_scale(data_stars['MC'])
data_stars['STDC'] = preprocessing.minmax_scale(data_stars['STDC'])
data_stars['EKC'] = preprocessing.minmax_scale(data_stars['EKC'])
data_stars['SC'] = preprocessing.minmax_scale(data_stars['SC'])
data_stars

,MIP,STDIP,EKIP,SIP,MC,STDC,EKC,SC
0,0.996764,0.537293,0.071732,0.017032,0.008473,0.050428,0.440305,0.195878
1,0.012521,0.540199,0.511476,0.226462,0.066817,0.300425,0.208266,0.024745
2,0.995639,0.512213,0.103070,0.019793,0.006066,0.039935,0.492970,0.251100
3,0.002532,0.152355,0.670838,0.558676,0.465657,0.782845,0.076236,0.000645
4,0.077237,0.241577,0.598744,0.388349,0.072318,0.352593,0.194257,0.019738
...,...,...,...,...,...,...,...,...
197,0.092712,0.265019,0.685468,0.470430,0.380366,0.652939,0.102877,0.002637
198,0.995217,0.270124,0.052280,0.055928,0.010894,0.090766,0.426235,0.155450
199,0.994373,0.527876,0.057790,0.016788,0.000000,0.000000,1.000000,1.000000
200,0.052476,0.485652,0.640038,0.351812,0.304181,0.585277,0.112483,0.003945


Выборочное среднее для столбца MIP после нормировки

In [49]:
data_stars['MIP'].mean()

0.5206502783086413

Обучите модель логистической регрессии, используя полученную после нормировки выборку в качестве тренировочного набора данных. Используйте следующие параметры:

LogisticRegression(random_state=2019, solver='lbfgs')


In [53]:
X = data_stars.copy()
y = pd.DataFrame(data['TARGET']).values.ravel()
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(random_state=2019, solver='lbfgs').fit(X, y)

Классификация новой звезды с параметрами

[[0.539, 0.162, 0.188, 0.653, 0.796, 0.233, 0.397, 0.065]]

In [54]:
New_stars = [0.539, 0.162, 0.188, 0.653, 0.796, 0.233, 0.397, 0.065]
reg.predict_proba([New_stars])

array([[0.4218309, 0.5781691]])

Введите вероятность отнесения звезды к классу пульсар:

In [ ]:
0.11814634

Классификация новой звезды, с помощью метода k-ближайших соседей, используя нормализованные данные выборки

In [55]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5, p=2)
neigh.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Введите расстояние от новой звезды до ближайшего соседа, используя евклидову метрику

In [56]:
neigh.kneighbors([New_stars])

(array([[0.89395099, 0.91711915, 0.92737999, 0.94232422, 0.94812764]]),
 array([[37, 71, 68,  6, 93]], dtype=int32))

Класс для новой звезды при k = 5 и евклидовой метрике

In [57]:
neigh.predict([New_stars])

array([1], dtype=int64)